In [90]:
%matplotlib nbagg
import pandas as pd
import os, sys
import numpy as np
from collections import defaultdict
from world_map_maker import create_world_map
#from caseDetection import detect_case
#import matplotlib as mpl
#mpl.use("GTK3cairo")
import matplotlib.pyplot as plt
#plt.plot([1,2,3,4],'*-')


In [ ]:
case1=[13,9,14,14,16,9,7,5,12,8,12,8,12,19,17,16,8,18,18,12,8,12,13,12,18,8,11,9,10,14,12,5,10,14,11,13,9,4,17,10,15,9,12,6,13,13,4,10,9,8,13,8,10,14,13,11,16,13,15,11,32,30,35,43,50,39,27,27,26,35,31,29,35,31,36,37,31,34,41,26,32,32,34,30,28,28,32,32,28,31,32,30,30,29,30,31,26,33,30,27,34,34,28,30,35,30,27,36,29,29,38,28,30,28,30,32,37,33,38,3]


# DATA ANALYSIS
- per country num obs
- per country num unique sIPs tested

#### reset index, as index were repeated due to concatenation earlier
#### this will not be a problem with any new dataset

In [58]:
df_all = pd.read_pickle("data_Apr17/case_detected_all_Apr17.pkl").reset_index()
df_sIP_subcat = pd.read_csv("data/Servers_IMC.txt")

### re-merge with df_all in case Servers_IMC was updated ...

In [60]:
df_all = df_all.merge(df_sIP_subcat, on=['sIP'])
df_all[df_all['subcat'].isnull()]

,index,ipids,ts,diff_list,retransmit_times,gIP,sIP,port,k1,k2,...,diff_p2,domain_x,subcat_x,case,intervention,pvalue,domain_y,subcat_y,domain,subcat


In [61]:
SAMPLENAME = 'Snapshot1'
RESULTS = "results/" + SAMPLENAME + "/"
if not os.path.exists(RESULTS):
    os.makedirs(RESULTS)

### SANITIZE ()
- get only countries with more than 400 unique sIPs
- select only one measurement for each unique sIPs in all countries

#### UNBIASED: so choose only good measurements...

In [62]:
df_all.columns

Index([u'index', u'ipids', u'ts', u'diff_list', u'retransmit_times', u'gIP', u'sIP', u'port', u'k1', u'k2', u'country', u'first_ts', u'diff_p1', u'diff_p2', u'domain_x', u'subcat_x', u'case', u'intervention', u'pvalue', u'domain_y', u'subcat_y', u'domain', u'subcat'], dtype='object')

In [63]:
df_all[df_all.case.isin([4,0])].to_csv('data_Apr17/Snapshot1_case04.csv')
df_good=df_all

In [64]:
# meaningful num of countries with at least 400 unique SIPs
THRESHOLD = 350

# per country, how many unique sIPs
df_unique_sIP_per_country = df_good.groupby(['country'])['sIP'].unique().apply(lambda x:len(x))
df_current = df_unique_sIP_per_country[df_unique_sIP_per_country > THRESHOLD]
df_current

country
AE         369
AF         497
AL         366
AM         370
AO         493
AR         434
AT         392
AZ         431
BA         492
BD         460
BE         431
BF         485
BG         472
BH         497
BI         495
...
TN         434
TR         447
TT         489
TW         387
TZ         394
UA         455
UG         480
US         357
UY         396
UZ         495
VE         442
VN         432
ZA         377
ZM         460
ZW         445
Name: sIP, Length: 139, dtype: int64

In [65]:
valid_countries = df_current.index
TOTAL_NUM_VALID_COUNTRIES = len(valid_countries)
print valid_countries, TOTAL_NUM_VALID_COUNTRIES

# filter df_good to only valid countries
# remove extra columns
df_valid = df_good[  df_good['country'].isin(valid_countries)   ] [['sIP', 'gIP', 'country', 'diff_list', 'domain', 'subcat', 'case']]

Index([u'AE', u'AF', u'AL', u'AM', u'AO', u'AR', u'AT', u'AZ', u'BA', u'BD', u'BE', u'BF', u'BG', u'BH', u'BI', u'BJ', u'BO', u'BW', u'BY', u'CA', u'CD', u'CG', u'CH', u'CI', u'CL', u'CM', u'CN', u'CO', u'CR', u'CU', u'CZ', u'DK', u'DO', u'DZ', u'EC', u'EE', u'EG', u'ES', u'FI', u'FR', u'GA', u'GB', u'GE', u'GH', u'GM', u'GQ', u'GR', u'GT', u'HK', u'HN', u'HR', u'HT', u'HU', u'ID', u'IE', u'IL', u'IN', u'IQ', u'IR', u'IT', u'JM', u'JO', u'JP', u'KE', u'KG', u'KH', u'KR', u'LA', u'LK', u'LR', u'LS', u'LT', u'LV', u'LY', u'MA', u'MD', u'MG', u'MK', u'ML', u'MM', u'MR', u'MU', u'MW', u'MX', u'MY', u'MZ', u'NA', u'NE', u'NG', u'NI', u'NL', u'NO', u'NP', u'NZ', u'OM', u'PA', u'PE', u'PG', u'PH', u'PK', ...], dtype='object') 139


In [66]:
df_good_sIP = df_valid
gp_good_sIP_country = df_good_sIP.groupby(['sIP','country'])

# even though there are multiple cases per sIP, country pair
# select only one
df_per_sIP_country = gp_good_sIP_country['case'].unique().reset_index()
df_per_sIP_country['count'] = gp_good_sIP_country['case'].count().reset_index()['case']
df_per_sIP_country.head()


,sIP,country,case,count
0,1.209.122.23,AE,[2],1
1,1.209.122.23,AF,[2],1
2,1.209.122.23,AL,[2],1
3,1.209.122.23,AM,[2],1
4,1.209.122.23,AO,[2],1


In [67]:
df_per_sIP_country[df_per_sIP_country['count']>1]

,sIP,country,case,count
898,104.20.28.30,BA,[3],2
1942,107.21.200.171,IL,[2],2
2860,109.169.47.177,KE,[1],2
3452,12.129.242.30,JP,[2],2
5351,129.35.201.80,SV,[0],2
6234,134.213.2.238,TN,[2],2
7832,141.211.243.44,CO,[4],2
8810,152.106.6.100,DZ,[2],2
9561,157.166.239.102,EE,[2],2
10479,161.58.179.241,SA,[2],2


# UNBIASED DATA
- filter df_good_sIP to only the random indices selected
- code under this can be run on current measurements as every sIP, country has just 1 measurement

In [68]:
# each country, sIP has just one entry in df_unbiased
df_unbiased=df_valid

In [69]:
df_temp = df_unbiased.groupby(['sIP', 'country'])['case'].apply(lambda x: len(x))
df_temp[df_temp > 1]

sIP              country
104.20.28.30     BA         2
107.21.200.171   IL         2
109.169.47.177   KE         2
12.129.242.30    JP         2
129.35.201.80    SV         2
134.213.2.238    TN         2
141.211.243.44   CO         2
152.106.6.100    DZ         2
157.166.239.102  EE         2
161.58.179.241   SA         2
163.166.156.201  JP         2
166.123.218.220  PT         2
169.229.216.200  SD         2
172.224.251.210  TW         2
172.233.7.37     TM         2
...
5.57.16.220     ZW         2
50.17.211.188   MY         2
54.174.93.70    PY         2
54.225.69.238   FI         2
54.80.7.219     TG         2
63.245.215.20   SO         2
64.14.32.140    TN         2
64.70.72.166    UG         2
66.211.169.3    US         2
68.168.105.234  KE         2
69.147.76.173   MU         2
69.28.53.205    LR         2
72.166.186.151  CG         2
74.113.188.100  EC         2
74.125.28.104   GH         2
Name: case, Length: 63, dtype: int64

In [70]:
grouped_by_country_subcat_case = df_unbiased.groupby(['country', 'subcat', 'case'])
count_per_country_subcat_case = grouped_by_country_subcat_case.count()
count_per_country_subcat_case.head(10)

sIP  gIP  diff_list  domain
country subcat   case                             
AE      adult    0       4    4          4       4
                 1       2    2          2       2
                 2      18   18         18      18
                 3       2    2          2       2
                 4       2    2          2       2
        arts     0       6    6          6       6
                 1       1    1          1       1
                 2      19   19         19      19
                 4       3    3          3       3
        business 0       7    7          7       7

In [71]:
# replace nan with 0
df_case_count = count_per_country_subcat_case.unstack(level=-1)['sIP'].fillna(0)
df_case_count['total'] = df_case_count.sum(axis=1)
df_case_count.to_html(RESULTS+'cases_per_country_subcat-unbiased-CenP.htm')
df_case_count.head()

case               0  1   2  3  4  total
country subcat                          
AE      adult      4  2  18  2  2     28
        arts       6  1  19  0  3     29
        business   7  0   7  0  4     18
        circum     0  0   9  1  1     11
        computers  4  1  11  5  1     22

## SAVE AS PICKLE TO MAKE PLOTS

In [98]:
df_case_count.to_pickle(RESULTS + "case_count_Snapshot1.pkl")

#### Censorship by country for each subcat
- create num(subcat) bar plots comparing case2 with (case1 + case3)

In [72]:
# clear all except df_case_count
#del df, df_bad, df_case_default, df_current, df_good, df_merged_unique_cases, df_obs_per_country, df_redo
#del df_sIP_CO, df_temp, df_unique, df_unique_sIP_per_country

In [73]:
group_by_subcat = df_case_count.reset_index().groupby('subcat')

### Per Subcategory Analysis, Excluding error cases

In [74]:
def get_ratios(subcat, indices, df_case_count):
    #if subcat == 'all':
        
    temp_df = df_case_count.ix[ group_by_subcat.groups[subcat] ].reset_index()
    temp_df['censored'] = temp_df[1]+temp_df[3]
    #temp_df['unknown'] = temp_df[-1]+temp_df[0]+temp_df[4]
    #temp_df = temp_df.rename(columns={2:'uncensored'})[['country', 'censored', 'uncensored', 'unknown']].set_index('country')
    temp_df = temp_df.rename(columns={2:'uncensored'})[['country', 'censored', 'uncensored']].set_index('country')

    # calculate ratios
    temp_df['total'] = temp_df.sum(axis=1)
    temp_df['ratio-censored'] = temp_df['censored']/(temp_df['total'])
    temp_df['ratio-uncensored'] = temp_df['uncensored']/(temp_df['total'])
    #temp_df['ratio-unknown'] = temp_df['unknown']/(temp_df['total'])

    # replace no entries with 0
    #df_country_case = temp_df.fillna(0)
    return temp_df

In [75]:
country_case_per_subcat = defaultdict(int)

# iterate over groupby object, indices
for subcat, indices in group_by_subcat.groups.iteritems():
    country_case_per_subcat[subcat] = get_ratios(subcat, indices, df_case_count)
    print subcat, len(country_case_per_subcat[subcat])
    
df_country_case_subcat = pd.concat(country_case_per_subcat)

science 139
arts 139
regional 139
shopping 139
reference 139
business 139
world 139
kids_and_teens 139
computers 139
recreation 139
sports 139
society 139
games 139
adult 139
home 139
health 139
news 139
vpn 139
circum 139


In [76]:
df_country_case_subcat.head(10)

case           censored  uncensored  total  ratio-censored  ratio-uncensored
      country                                                               
adult AE              4          18     22        0.181818          0.818182
      AF              7          28     35        0.200000          0.800000
      AL              4           9     13        0.307692          0.692308
      AM              1          16     17        0.058824          0.941176
      AO              5          30     35        0.142857          0.857143
      AR              2          28     30        0.066667          0.933333
      AT              3          23     26        0.115385          0.884615
      AZ              3          25     28        0.107143          0.892857
      BA              3          30     33        0.090909          0.909091
      BD              3          26     29        0.103448          0.896552

In [77]:
all_results = list(df_country_case_subcat.columns)
print all_results

['censored', 'uncensored', 'total', 'ratio-censored', 'ratio-uncensored']


In [78]:
censorship_ratio = defaultdict(int)
for key in all_results:
    censorship_ratio[key] = df_country_case_subcat[key].unstack(0)
    censorship_ratio[key].to_html(RESULTS+'censorship_'+key+'.html')

In [79]:
censorship_ratio['ratio-censored']

,adult,arts,business,circum,computers,games,health,home,kids_and_teens,news,recreation,reference,regional,science,shopping,society,sports,vpn,world
country,,,,,,,,,,,,,,,,,,,
AE,0.181818,0.050000,0.000000,0.100000,0.352941,0.222222,0.312500,0.150000,0.222222,0.142857,0.285714,0.187500,0.250000,0.285714,0.250000,0.388889,0.083333,0.272727,0.272727
AF,0.200000,0.205882,0.052632,0.157895,0.241379,0.321429,0.250000,0.083333,0.142857,0.230769,0.214286,0.114286,0.333333,0.142857,0.120000,0.257143,0.050000,0.214286,0.172414
AL,0.307692,0.210526,0.214286,0.000000,0.571429,0.500000,0.250000,0.500000,0.125000,0.454545,0.461538,0.222222,0.142857,0.200000,0.333333,0.272727,0.222222,0.428571,0.133333
AM,0.058824,0.083333,0.071429,0.000000,0.100000,0.250000,0.250000,0.125000,0.000000,0.100000,0.285714,0.000000,0.125000,0.260870,0.062500,0.260870,0.153846,0.333333,0.142857
AO,0.142857,0.093750,0.105263,0.157895,0.233333,0.227273,0.320000,0.090909,0.071429,0.076923,0.076923,0.117647,0.153846,0.171429,0.045455,0.272727,0.052632,0.166667,0.129032
AR,0.066667,0.148148,0.076923,0.058824,0.190476,0.240000,0.173913,0.100000,0.076923,0.076923,0.206897,0.071429,0.100000,0.031250,0.153846,0.241379,0.052632,0.300000,0.130435
AT,0.115385,0.080000,0.076923,0.125000,0.285714,0.285714,0.157895,0.100000,0.083333,0.100000,0.181818,0.136364,0.000000,0.160000,0.125000,0.250000,0.062500,0.416667,0.160000
AZ,0.107143,0.074074,0.200000,0.133333,0.157895,0.238095,0.272727,0.176471,0.166667,0.230769,0.291667,0.037037,0.181818,0.259259,0.222222,0.285714,0.166667,0.166667,0.178571
BA,0.090909,0.090909,0.000000,0.105263,0.214286,0.250000,0.185185,0.083333,0.066667,0.230769,0.107143,0.033333,0.066667,0.181818,0.043478,0.200000,0.047619,0.214286,0.096774


## PLOT CENSORSHIP

#### games category

In [80]:
df = df_unbiased[ df_unbiased['subcat']=='games' ]
game_censorship = pd.DataFrame( df.groupby(['case', 'domain'])['country'].unique() )
game_censorship['count'] = game_censorship['country'].apply( lambda x: len(x) )
game_censorship['country'] = game_censorship['country'].apply( lambda x: np.sort(x) )

In [81]:
game_censorship

country  \
case domain                                                                      
0    addictinggames.com                   [BE, BF, GT, MK, MW, NZ, PS, SN, US]   
     battle.net                                           [BE, BJ, NO, PK, PT]   
     boardgamegeek.com                                        [BE, EE, RU, SG]   
     bungie.net                               [CG, EC, GE, HT, MX, NA, PE, US]   
     chess.com               [AE, BE, BO, EC, FR, GE, HR, KH, LR, MY, MZ, S...   
     curse.com                                                [AE, PA, PE, US]   
     dorkly.com                                       [GE, GT, LV, NI, SE, TM]   
     ea.com                                       [AO, CN, DO, LV, ML, MU, PS]   
     freelotto.com                        [BO, CL, FR, GT, IL, SA, SV, TN, ZW]   
     gamesradar.com                                           [HT, NI, PL, US]   
     king.com                                         [CD, DO, GR, HK, LK, SG]   
     kongregate.com                                                       [TL]   
     leagueoflegends.com                          [BE, CH, CI, EC, HN, HT, SI]   
     minecraft.net                                                        [MD]   
     minecraftforum.net                                               [EC, ES]   
     miniclip.com                                                 [BE, FR, MX]   
     mmo-champion.com                                 [BD, GT, MA, PA, PE, SI]   
     n4g.com                 [AL, AM, AT, BJ, CL, CU, EG, GE, MM, MX, NO, T...   
     nintendo.com                                     [AE, CA, CL, DO, MU, RU]   
     pcgamer.com                                                  [AO, CI, FR]   
     pch.com                 [AE, ES, GB, HR, IE, JM, JO, LR, MK, RO, RW, T...   
     planetminecraft.com                                          [CG, MU, MX]   
     rockstargames.com       [AL, BE, BO, GB, GR, GT, JO, KR, LR, NO, SN, S...   
     square-enix.com                  [BA, BH, CG, DO, GB, LR, MK, PG, UA, UZ]   
     store.steampowered.com  [BE, BI, BO, CD, CG, HT, IT, JO, MM, NG, PK, R...   
     williamhill.com                                                      [UA]   
     wizards.com                              [AO, CI, CU, HR, IL, MM, PE, TN]   
     wowhead.com                                          [CL, DO, ML, RU, SE]   
     xbox.com                 [CA, CG, CU, GE, GQ, IL, JM, JP, LR, MK, PT, VN]   
1    battle.net                                       [CU, ES, MM, PG, SD, SY]   
...                                                                        ...   
3    wizards.com                                                  [BE, LR, LS]   
     wowhead.com                                                      [LR, SY]   
     xbox.com                                                             [RS]   
4    addictinggames.com                       [AL, AO, CA, NP, PG, RO, RW, TW]   
     battle.net                               [AL, CH, EC, IN, MA, NP, RW, TM]   
     boardgamegeek.com                                        [IL, NA, NZ, TW]   
     bungie.net                   [AT, AZ, GB, IR, LR, LT, LY, MD, NL, NP, RW]   
     chess.com                                        [CU, GB, HN, RU, SY, ZA]   
     curse.com                                                [AL, LY, PT, TN]   
     dorkly.com                                                   [HN, MW, NE]   
     ea.com                                           [EG, HN, IL, IR, RW, SZ]   
     freelotto.com                                        [AE, IN, NP, RW, SY]   
     gamesradar.com                                       [AM, AT, IL, SK, VN]   
     kongregate.com          [AL, CD, CH, DO, FI, GT, MY, MZ, NL, PG, RO, T...   
     leagueoflegends.com                                  [AO, IL, PG, RS, SL]   
     minecraftforum.net                                       [GQ, IL, MY, NZ]   
     miniclip.com                                    

In [82]:
game_censorship.to_html(RESULTS+subcat+'_censorship.html')

### Every subcat


In [83]:
print df_unbiased['subcat'].unique()

['health' 'society' 'recreation' 'world' 'arts' 'circum' 'adult' 'news'
 'science' 'sports' 'home' 'reference' 'business' 'regional'
 'kids_and_teens' 'computers' 'games' 'shopping' 'vpn']


In [84]:
df_unbiased[ df_unbiased['subcat'].isnull() ]['sIP'].unique()

array([], dtype=object)

In [85]:
for subcat in df_unbiased['subcat'].unique():
    df = df_unbiased[ df_unbiased['subcat'] == subcat ]
    censorship = pd.DataFrame( df.groupby(['case', 'domain'])['country'].unique() )
    censorship['count'] = censorship['country'].apply( lambda x: len(x) )
    censorship['country'] = censorship['country'].apply( lambda x: np.sort(x) )
    censorship.to_html(RESULTS + 'subcat_censorship_'+subcat+'.html')

In [86]:
censorship

country  \
case domain                                                                  
0    earthvpn.com                                             [FR, NZ, SE]   
     express-vpn.com                                      [AL, MM, MR, NO]   
     get.ironsocket.com                                   [CG, PA, PE, SE]   
     hidemyass.com                                            [NI, RU, SG]   
     hsselite.com                                                 [PY, RS]   
     ibvpn.com                                [CG, CI, CM, JO, MD, PH, SN]   
     ipvanish.com                         [BF, CL, HT, MK, MM, NP, SI, TW]   
     nordvpn.com                                                  [PA, PY]   
     proxify.com                                              [HT, NI, SK]   
     purevpn.com                                  [JP, LV, TM, VE, VN, ZA]   
     safervpn.com                             [AO, BG, NP, RS, UG, UY, VE]   
     strongvpn.com                            [CU, ES, HT, JO, LV, MM, NI]   
     thesafety.us                                         [DK, HR, IT, TH]   
     tunnelbear.com                                           [EC, IN, SN]   
     vpnintouch.com               [BW, CN, DK, GB, IL, LK, NG, NI, PL, ZW]   
1    hidemyass.com                                            [AL, MX, PG]   
     hsselite.com                                                     [NG]   
     ibvpn.com                                                        [TR]   
     nordvpn.com                                                      [PK]   
     purevpn.com                                                  [AT, US]   
     strongvpn.com                                                    [PY]   
     thesafety.us                                             [CA, KE, RS]   
     tunnelbear.com                                                   [RW]   
     vpnintouch.com                                                   [VE]   
2    earthvpn.com        [CA, CN, CR, DK, GB, ID, IE, IL, IN, IQ, IR, I...   
     express-vpn.com     [AF, AM, AO, AR, BA, BD, BE, BF, BG, BH, BI, B...   
     hidemyass.com       [AE, AF, AM, AO, AT, BA, BD, BF, BG, BH, BI, B...   
     hsselite.com        [AE, AF, AO, AR, AT, BA, BE, BF, BG, BH, BI, B...   
     ibvpn.com           [AF, AL, AM, AO, AR, BA, BD, BE, BG, BH, BI, B...   
     ipvanish.com        [AE, AF, AM, AO, AZ, BA, BD, BH, BI, BJ, BW, B...   
...                                                                    ...   
     vpnintouch.com      [AE, AF, AL, AM, AO, AR, AT, AZ, BA, BD, BE, B...   
3    earthvpn.com                                                     [LR]   
     express-vpn.com                                              [ID, LR]   
     get.ironsocket.com  [AE, AF, AL, AM, AO, AR, AT, AZ, BA, BD, BF, B...   
     hidemyass.com                                        [BE, LR, MD, NL]   
     hsselite.com                                                 [IR, LR]   
     ibvpn.com                                                    [BF, ID]   
     ipvanish.com                                                 [BE, RS]   
     nordvpn.com         [AE, AF, AM, AR, AT, BA, BD, BF, BG, BH, BI, B...   
     purevpn.com                                          [CU, GA, GQ, LR]   
     safervpn.com                                                 [LR, MY]   
     strongvpn.com                                                    [LR]   
     thesafety.us                                                 [CO, LR]   
     tunnelbear.com      [AE, AF, AL, AM, AO, AR, AT, BA, BD, BE, BF, B...   
     vpnintouch.com                                                   [CH]   
4    earthvpn.com                                         [LS, MD, RW, TN]   
     express-vpn.com                                  [AE, EC, NP, RU, SE]   
     get.ironsocket.com                                           [FR, IN]   
     hidemyass.com                [AR, EC, GT, HR, IN, IR, MY, PK, RW, TT]   
     hsseli

# WORLD MAP

- ignore the subcategories
- plot TOTAL censorship for every country

In [87]:
df = pd.DataFrame(df_unbiased.groupby(['country', 'case'])['sIP'].count()).unstack().reset_index().set_index('country')['sIP']
df['total'] = df.sum(axis=1)
df['case1'] = df[1]/df['total']
df['case2'] = df[2]/df['total']
df['case3'] = df[3]/df['total']
case1 = df['case1'].to_dict()
case2 = df['case2'].to_dict()
case3 = df['case3'].to_dict()

In [88]:
df

case,0,1,2,3,4,total,case1,case2,case3
country,,,,,,,,,
AE,63,27,212,33,35,370,0.072973,0.572973,0.089189
AF,17,36,390,53,1,497,0.072435,0.784708,0.106640
AL,51,60,175,13,67,366,0.163934,0.478142,0.035519
AM,21,12,268,34,35,370,0.032432,0.724324,0.091892
AO,30,19,388,49,7,493,0.038540,0.787018,0.099391
AR,21,12,343,40,18,434,0.027650,0.790323,0.092166
AT,24,16,298,41,14,393,0.040712,0.758270,0.104326
AZ,15,34,306,36,40,431,0.078886,0.709977,0.083527
BA,15,6,410,52,10,493,0.012170,0.831643,0.105477


In [94]:
create_world_map(case1, "Server to Client Censorship", True)

0.0040404040404 0.441926345609
http://chart.apis.google.com/chart?cht=map:fixed=-60,-20,80,-35&chs=600x400&chma=0,60,0,0&chld=AA|AA|AA|AA|AA|AA|SZ|GM|TG|SO|BW|CM|TT|CR|SL|GA|CD|DZ|SV|CI|TZ|GR|SI|UG|FR|SE|ZM|PS|BH|BY|UZ|BA|GH|BF|SA|DK|HT|LA|SK|LK|ML|NE|GT|RS|JP|SY|NA|OM|TW|ZW|JO|MG|LS|MM|MW|TL|EC|SD|BI|SN|PH|IQ|EG|IT|CU|LT|DO|BG|EE|PY|BD|UY|MU|CL|KE|LY|RO|PK|MR|MK|NI|PA|KR|MY|PE|UA|KH|AR|CZ|MZ|LR|NO|NP|HR|CO|SG|MX|NG|AM|GB|PT|MA|GQ|KG|IR|IN|HN|BE|VN|LV|NZ|RU|PL|AO|ID|AT|HU|BO|ES|TH|HK|VE|ZA|FI|NL|BJ|IE|CA|TN|MD|CH|TR|JM|IL|RW|AF|AE|CN|AZ|US|GE|CG|AL|TM|PG&chco=808080|808080|c6dbef|9ecae1|6baed6|3182bd|08519c|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|

In [95]:
create_world_map(case3, "Client to Server Censorship", True)

0.0355191256831 0.546840958606
http://chart.apis.google.com/chart?cht=map:fixed=-60,-20,80,-35&chs=600x400&chma=0,60,0,0&chld=AA|AA|AA|AA|AA|AA|AL|PA|PG|RW|TM|PE|CG|HN|TN|GE|VN|GQ|IL|TT|DO|TL|AZ|CH|ZA|GT|TG|HK|PH|IQ|GA|CD|AE|ES|BJ|GB|TH|PK|MM|OM|CO|KG|TZ|MX|AM|AR|KH|BW|NI|VE|BI|HR|SL|DK|ZW|LK|SY|CZ|PT|ZM|IT|UA|NG|PY|SN|JP|FI|JM|KR|TW|EC|TR|BO|SO|HT|CA|MA|AO|IN|FR|UG|GH|NE|SV|MR|EE|CM|IE|SG|MG|KE|BD|SK|EG|ID|NZ|RO|AT|DZ|BY|RU|LV|BA|CI|NP|MZ|GR|NA|BH|SD|AF|CN|CR|SA|CL|NL|ML|LS|MU|MW|US|SZ|NO|LA|LY|GM|SI|PS|LT|PL|BE|SE|UZ|BF|HU|CU|MY|BG|MK|JO|IR|MD|RS|UY|LR&chco=808080|808080|c6dbef|9ecae1|6baed6|3182bd|08519c|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|c6dbef|

In [96]:
create_world_map(case2, "No Censorship", True)

0.228758169935 0.881287726358
http://chart.apis.google.com/chart?cht=map:fixed=-60,-20,80,-35&chs=600x400&chma=0,60,0,0&chld=AA|AA|AA|AA|AA|AA|LR|PG|AL|TM|AE|RW|CG|IL|BE|HN|BO|MD|GE|US|HR|TN|BJ|DO|NL|NZ|CN|ZA|MK|CU|ES|PL|IE|SG|IR|CH|RU|GQ|AZ|NP|IN|MA|SY|TR|HT|UY|GB|HU|AM|JM|FI|IT|KG|NO|CL|LV|NG|CO|MM|NI|KH|CZ|MX|SE|JO|MY|VE|HK|PY|KR|ID|PK|CA|NA|PT|ZW|AT|SN|MR|UA|TH|TW|EC|SV|GT|SL|LK|ML|BF|RS|FR|RO|UZ|AF|VN|BG|AO|SK|SI|LY|AR|MW|LT|MZ|PE|MU|GR|JP|UG|CI|KE|LA|SA|BD|EE|EG|IQ|PH|TT|BY|GH|BH|DZ|BA|CM|LS|OM|SZ|PS|DK|ZM|NE|BW|SD|BI|MG|TZ|CR|PA|SO|GM|CD|TG|TL|GA&chco=808080|808080|c6dbef|9ecae1|6baed6|3182bd|08519c|c6dbef|c6dbef|9ecae1|9ecae1|9ecae1|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6baed6|6